# Importing libraries

In [46]:
# Run if not everything installed
!pip install tensorflow     > null 2>&1
!pip install seaborn        > null 2>&1
!pip install numpy          > null 2>&1
!pip install pillow         > null 2>&1
!pip install opencv-python  > null 2>&1
!pip install scikit-learn   > null 2>&1
#!pip install torchvision    > null 2>&1

In [1]:
#import tensorflow as tf
#import seaborn as sns
import numpy as np

from PIL import Image
import glob
from collections import defaultdict
from tensorflow import keras
from tensorflow.keras import layers, models, optimizers

from sklearn.metrics import mean_squared_error, roc_auc_score, classification_report

# from torchvision.transforms import (
#     Compose,
#     RandomHorizontalFlip,
#     RandomRotation,
# )

# Preprocessing

In [13]:
# Specifying final image size and resizing to that size
IMG_SIZE = (94, 125)
def pixels_from_path(file_path):
    im = Image.open(file_path)
    im = im.convert("RGB")
    im = im.resize(IMG_SIZE)
    np_im = np.array(im)
    # Returns 3D array of RGB values for image
    return np_im

In [3]:
# Input relevant filepath before '/cat'
cat_filepath = 'cat/*'
dog_filepath = 'dog/*'
tiger_filepath = 'tiger/*'
lion_filepath = 'lion/*'

In [4]:
shape_counts = defaultdict(int)
for i, cat in enumerate(glob.glob(cat_filepath)):
    # Will print iteration no. if i+1 is a multiple of 500
    if (i+1)%500==0:
        print(i)
    img_shape = pixels_from_path(cat).shape
    shape_counts[str(img_shape)]= shape_counts[str(img_shape)]+ 1

499
999
1499
1999
2499
2999
3499
3999
4499
4999
5499
5999
6499
6999
7499
7999
8499
8999
9499
9999
10499
10999
11499
11999
12499


In [5]:
shape_items = list(shape_counts.items())
shape_items.sort(key = lambda x: x[1])
shape_items.reverse()

In [6]:
# 10% of the data will be used for validation
validation_size = 0.1
img_size = IMG_SIZE # resize images to be 0.25x most common shape (374x500)
num_channels = 3 # RGB
sample_size = 25000 # Using all training data for the sample size

In [7]:
len(glob.glob(cat_filepath))

12500

In [8]:
len(glob.glob(lion_filepath))

2621

In [9]:
len(glob.glob(tiger_filepath))

2480

In [10]:
pixels_from_path(glob.glob(cat_filepath)[5]).shape

(125, 94, 3)

In [11]:
# Training size
SAMPLE_SIZE = 11250
SAMPLE_SIZE_TUNING = 2200    # different due to different dataset size

# Validation size
valid_size = 1250
valid_size_tuning = 220      # different due to different dataset size

In [37]:
print("loading training cat images...")
cat_train_set = np.asarray([
    pixels_from_path(cat)
    for cat in glob.glob(cat_filepath)[:SAMPLE_SIZE]
    if pixels_from_path(cat) is not None
])

print("loading training dog images...")
dog_train_set = np.asarray([
    pixels_from_path(dog)
    for dog in glob.glob(dog_filepath)[:SAMPLE_SIZE]
    if pixels_from_path(dog) is not None
])

print("loading training tiger images...")
tiger_train_set = np.asarray([
    pixels_from_path(tiger)
    for tiger in glob.glob(tiger_filepath)[:SAMPLE_SIZE_TUNING]
    if pixels_from_path(tiger) is not None
])

print("loading training lion images...")
lion_train_set = np.asarray([
    pixels_from_path(lion)
    for lion in glob.glob(lion_filepath)[:SAMPLE_SIZE_TUNING]
    if pixels_from_path(lion) is not None
])

loading training cat images...
loading training dog images...
loading training tiger images...
loading training lion images...


C:\Users\Rachel\IdeaProjects\DLE305-Assessment-2\.venv\Lib\site-packages\PIL\Image.py:1054: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


In [15]:
print("loading validation cat images...")
cat_valid_set = np.asarray([
    pixels_from_path(cat)
    for cat in glob.glob(cat_filepath)[-valid_size:]
    if pixels_from_path(cat) is not None
])

print("loading validation dog images...")
dog_valid_set = np.asarray([
    pixels_from_path(dog)
    for dog in glob.glob(dog_filepath)[-valid_size:]
    if pixels_from_path(dog) is not None
])

print("loading training tiger images...")
tiger_valid_set = np.asarray([
    pixels_from_path(tiger)
    for tiger in glob.glob(tiger_filepath)[-valid_size_tuning:]
    if pixels_from_path(tiger) is not None
])

print("loading training lion images...")
lion_valid_set = np.asarray([
    pixels_from_path(lion)
    for lion in glob.glob(lion_filepath)[-valid_size_tuning:]
    if pixels_from_path(lion) is not None
])

loading validation cat images...
loading validation dog images...
loading training tiger images...
loading training lion images...


In [16]:
x_train = np.concatenate([cat_train_set, dog_train_set])
# Applying labels based on sample size because data are currently ordered by class
labels_train = np.asarray([1 for _ in range(SAMPLE_SIZE)]+[0 for _ in range(SAMPLE_SIZE)])

In [38]:
x_tune = np.concatenate([tiger_train_set, lion_train_set])
# Applying labels based on sample size because data are currently ordered by class
labels_tune = np.asarray([1 for _ in range(SAMPLE_SIZE_TUNING)]+[0 for _ in range(SAMPLE_SIZE_TUNING)])

In [18]:
x_valid = np.concatenate([cat_valid_set, dog_valid_set])
# Applying labels based on validation sample size because data are currently ordered by class
labels_valid = np.asarray([1 for _ in range(valid_size)]+[0 for _ in range(valid_size)])

In [19]:
x_valid_tune = np.concatenate([tiger_valid_set, lion_valid_set])
# Applying labels based on validation sample size because data are currently ordered by class
labels_valid_tune = np.asarray([1 for _ in range(valid_size_tuning)]+[0 for _ in range(valid_size_tuning)])

In [20]:
# Reshape labels to match output
labels_train = labels_train.reshape(-1,1)
labels_valid = labels_valid.reshape(-1,1)

In [39]:
labels_tune = labels_tune.reshape(-1,1)
labels_valid_tune = labels_valid_tune.reshape(-1,1)

In [22]:
x_train.shape

(22500, 125, 94, 3)

In [23]:
labels_train.shape

(22500, 1)

In [24]:
labels_train[:10]  # Checking values to ensure they're not None

array([[1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1]])

In [25]:
labels_train[22490:]  # Checking values to ensure they're not None

array([[0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0]])

In [26]:
# Fully connected layer neuron number
fc_layer_size = 256

# CNN from A2

In [27]:
# Convolution parameters
conv_inputs = keras.Input(shape=(img_size[1], img_size[0],3), name='ani_image')
conv_layer = layers.Conv2D(128, kernel_size=3, activation='relu')(conv_inputs)
conv_layer = layers.MaxPool2D(pool_size=(2,2))(conv_layer)

conv_layer = layers.Conv2D(128, kernel_size=3, activation='relu')(conv_layer)
conv_layer = layers.MaxPool2D(pool_size=(2,2))(conv_layer)

conv_x = layers.Flatten(name = 'flattened_features')(conv_layer) #turn image to vector.

conv_x = layers.Dense(fc_layer_size, activation='relu', name='first_layer')(conv_x)
conv_x = layers.Dense(fc_layer_size, activation='relu', name='second_layer')(conv_x)
conv_outputs = layers.Dense(1, activation='sigmoid', name='class')(conv_x)

catdog_model = keras.Model(inputs=conv_inputs, outputs=conv_outputs)

In [28]:
customAdam = keras.optimizers.Adam(learning_rate=1e-6)
catdog_model.compile(optimizer=customAdam,  # Optimizer
                        # Loss function to minimize
                        loss="BinaryCrossentropy",
                        # List of metrics to monitor
                        metrics=["BinaryCrossentropy","MeanSquaredError", "accuracy"])

In [29]:
print('# Fit model on training data')

history = catdog_model.fit(x_train,
                              labels_train,
                              batch_size=64,
                              shuffle = True,
                              epochs=30,
                              validation_data=(x_valid, labels_valid))

# Fit model on training data
Epoch 1/30
352/352 ━━━━━━━━━━━━━━━━━━━━ 172s 485ms/step - BinaryCrossentropy: 1.4450 - MeanSquaredError: 0.3546 - accuracy: 0.5379 - loss: 1.4450 - val_BinaryCrossentropy: 0.8324 - val_MeanSquaredError: 0.2705 - val_accuracy: 0.6148 - val_loss: 0.8324
Epoch 2/30
352/352 ━━━━━━━━━━━━━━━━━━━━ 171s 485ms/step - BinaryCrossentropy: 0.7920 - MeanSquaredError: 0.2599 - accuracy: 0.6186 - loss: 0.7920 - val_BinaryCrossentropy: 0.7302 - val_MeanSquaredError: 0.2441 - val_accuracy: 0.6388 - val_loss: 0.7302
Epoch 3/30
352/352 ━━━━━━━━━━━━━━━━━━━━ 170s 484ms/step - BinaryCrossentropy: 0.6909 - MeanSquaredError: 0.2323 - accuracy: 0.6512 - loss: 0.6909 - val_BinaryCrossentropy: 0.6932 - val_MeanSquaredError: 0.2321 - val_accuracy: 0.6412 - val_loss: 0.6932
Epoch 4/30
352/352 ━━━━━━━━━━━━━━━━━━━━ 168s 478ms/step - BinaryCrossentropy: 0.6443 - MeanSquaredError: 0.2171 - accuracy: 0.6742 - loss: 0.6443 - val_BinaryCrossentropy: 0.6702 - val_MeanSquaredError: 0.2243 - val

In [30]:
# Evaluating CNN model predictions on validation data

#preds = np.asarray(preds).flatten()
labels_flat = np.asarray(labels_valid).flatten()

preds = catdog_model.predict(x_valid)
preds = np.asarray([pred[0] for pred in preds])

np.corrcoef(preds, labels_flat)

79/79 ━━━━━━━━━━━━━━━━━━━━ 5s 59ms/step


array([[1.        , 0.57567404],
       [0.57567404, 1.        ]])

In [31]:
# Saving model
catdog_model.save('untuned_model.keras')

# Fine-tuning model on big cats

In [60]:
# Loading model to variable
untuned_model = keras.models.load_model('untuned_model.keras')

In [61]:
for layer in untuned_model.layers:
    if isinstance(layer, layers.Conv2D):
        layer.trainable = False

In [62]:
# Remove classification head
conv_x = untuned_model.get_layer("flattened_features").output

# Add new dense layers for big cats
conv_x = layers.Dense(fc_layer_size, activation='relu', name='new_fc')(conv_x)
conv_outputs = layers.Dense(1, activation='sigmoid', name='bigcat_class')(conv_x)

# Create new model
bigcat_model = keras.Model(inputs=untuned_model.input, outputs=conv_outputs)

In [63]:
customAdam = keras.optimizers.Adam(learning_rate=1e-6)  # Small learning rate for transfer learning
bigcat_model.compile(optimizer=customAdam,
                         loss="BinaryCrossentropy",
                         metrics=["BinaryCrossentropy","MeanSquaredError", "accuracy"])

In [64]:
history = bigcat_model.fit(x_tune,
                               labels_tune,
                               batch_size=64,
                               shuffle=True,
                               epochs=15,  # Start with fewer epochs
                               validation_data=(x_valid_tune, labels_valid_tune))

Epoch 1/15
69/69 ━━━━━━━━━━━━━━━━━━━━ 15s 215ms/step - BinaryCrossentropy: 2.8196 - MeanSquaredError: 0.3897 - accuracy: 0.5701 - loss: 2.8196 - val_BinaryCrossentropy: 1.5988 - val_MeanSquaredError: 0.2962 - val_accuracy: 0.6636 - val_loss: 1.5988
Epoch 2/15
69/69 ━━━━━━━━━━━━━━━━━━━━ 15s 219ms/step - BinaryCrossentropy: 1.4057 - MeanSquaredError: 0.2758 - accuracy: 0.6672 - loss: 1.4057 - val_BinaryCrossentropy: 1.3368 - val_MeanSquaredError: 0.2611 - val_accuracy: 0.7000 - val_loss: 1.3368
Epoch 3/15
69/69 ━━━━━━━━━━━━━━━━━━━━ 15s 212ms/step - BinaryCrossentropy: 1.1021 - MeanSquaredError: 0.2274 - accuracy: 0.7282 - loss: 1.1021 - val_BinaryCrossentropy: 1.1201 - val_MeanSquaredError: 0.2388 - val_accuracy: 0.7159 - val_loss: 1.1201
Epoch 4/15
69/69 ━━━━━━━━━━━━━━━━━━━━ 15s 214ms/step - BinaryCrossentropy: 0.8754 - MeanSquaredError: 0.1948 - accuracy: 0.7569 - loss: 0.8754 - val_BinaryCrossentropy: 1.0149 - val_MeanSquaredError: 0.2244 - val_accuracy: 0.7295 - val_loss: 1.0149
Epoc

In [65]:
# Evaluating finetuned model predictions on validation data
labels_flat = np.asarray(labels_valid_tune).flatten()
preds = bigcat_model.predict(x_valid_tune)
preds = np.asarray([pred[0] for pred in preds])

print(np.corrcoef(preds, labels_flat))

14/14 ━━━━━━━━━━━━━━━━━━━━ 1s 57ms/step
[[1.         0.60602215]
 [0.60602215 1.        ]]


In [66]:
# MSE for predictions
# Closer to 0 is better
print(mean_squared_error(labels_flat, preds))

0.17466860461105638


In [67]:
# Receiver Operating Characteristic and Area Under Curve
# Closer to 1 is better
print(roc_auc_score(labels_flat, preds))

0.8585743801652892


In [68]:
bigcat_model.save("tuned_model.keras")

Optional for larger datasets

In [69]:
for layer in bigcat_model.layers:
    if isinstance(layer, layers.Conv2D):
        layer.trainable = True  # Unfreeze convolutional layers

In [70]:
customAdam = keras.optimizers.Adam(learning_rate=1e-7)  # Even smaller learning rate
bigcat_model.compile(optimizer=customAdam,
                         loss="BinaryCrossentropy",
                         metrics=["BinaryCrossentropy","MeanSquaredError", "accuracy"])

history_fine = bigcat_model.fit(x_tune,
                                    labels_tune,
                                    batch_size=64,
                                    shuffle=True,
                                    epochs=5,       # Fewer epochs
                                    validation_data=(x_valid_tune, labels_valid_tune))

Epoch 1/5
69/69 ━━━━━━━━━━━━━━━━━━━━ 35s 490ms/step - BinaryCrossentropy: 0.2006 - MeanSquaredError: 0.0558 - accuracy: 0.9254 - loss: 0.2006 - val_BinaryCrossentropy: 0.7143 - val_MeanSquaredError: 0.1747 - val_accuracy: 0.7773 - val_loss: 0.7143
Epoch 2/5
69/69 ━━━━━━━━━━━━━━━━━━━━ 34s 491ms/step - BinaryCrossentropy: 0.1875 - MeanSquaredError: 0.0526 - accuracy: 0.9310 - loss: 0.1875 - val_BinaryCrossentropy: 0.7193 - val_MeanSquaredError: 0.1754 - val_accuracy: 0.7886 - val_loss: 0.7193
Epoch 3/5
69/69 ━━━━━━━━━━━━━━━━━━━━ 34s 486ms/step - BinaryCrossentropy: 0.1892 - MeanSquaredError: 0.0529 - accuracy: 0.9314 - loss: 0.1892 - val_BinaryCrossentropy: 0.7064 - val_MeanSquaredError: 0.1738 - val_accuracy: 0.7727 - val_loss: 0.7064
Epoch 4/5
69/69 ━━━━━━━━━━━━━━━━━━━━ 34s 490ms/step - BinaryCrossentropy: 0.1861 - MeanSquaredError: 0.0524 - accuracy: 0.9297 - loss: 0.1861 - val_BinaryCrossentropy: 0.7089 - val_MeanSquaredError: 0.1736 - val_accuracy: 0.7773 - val_loss: 0.7089
Epoch 5/

In [71]:
# Evaluating finetuned model predictions on validation data
labels_flat = np.asarray(labels_valid_tune).flatten()
preds = bigcat_model.predict(x_valid_tune)
preds = np.asarray([pred[0] for pred in preds])

print(np.corrcoef(preds, labels_flat))

14/14 ━━━━━━━━━━━━━━━━━━━━ 1s 58ms/step
[[1.        0.6089629]
 [0.6089629 1.       ]]


In [72]:
# MSE for predictions
# Closer to 0 is better
print(mean_squared_error(labels_flat, preds))

0.17388696923331834


In [73]:
# Receiver Operating Characteristic and Area Under Curve
# Closer to 1 is better
print(roc_auc_score(labels_flat, preds))

0.8604338842975207


In [74]:
bigcat_model.save("retuned_model.keras")